**Customer Segmentation with RFM**

In [1]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df_ = pd.read_excel("/kaggle/input/online-retails/online_retail_II.xlsx", sheet_name= "Year 2009-2010")
df = df_.copy()

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


In [2]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.dropna(inplace=True)
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom,30.00


In [3]:
### Calculating RFM Metrics ###

today_date = dt.datetime(2010, 12, 11)

rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: num.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ['recency', 'frequency', 'monetary']
rfm = rfm[(rfm["monetary"] > 0)]
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12347.00,3,2,1323.32
12348.00,74,1,222.16
12349.00,43,4,2646.99
12351.00,11,1,300.93
12352.00,11,2,343.80


In [4]:
### Calculating RFM Scores ###

rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])


rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12347.00,3,2,1323.32,5,2,4,52
12348.00,74,1,222.16,2,1,1,21
12349.00,43,4,2646.99,3,3,5,33
12351.00,11,1,300.93,5,1,2,51
12352.00,11,2,343.80,5,2,2,52


In [5]:
### Creating RFM Segments ###

seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions',
}



rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12347.00,3,2,1323.32,5,2,4,52,potential_loyalists
12348.00,74,1,222.16,2,1,1,21,hibernating
12349.00,43,4,2646.99,3,3,5,33,need_attention
12351.00,11,1,300.93,5,1,2,51,new_customers
12352.00,11,2,343.80,5,2,2,52,potential_loyalists


**Customer Lifetime Value**

In [6]:
#import datetime as dt
#import pandas as pd
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 30)
#pd.set_option('display.float_format', lambda x: '%.2f' % x)

#df_ = pd.read_excel("/kaggle/input/online-retails/online_retail_II.xlsx", sheet_name= "Year 2009-2010")
#df = df_.copy()

#df.head()

#df["TotalPrice"] = df["Quantity"] * df["Price"]
#df.dropna(inplace=True)
#df.head()

In [7]:
cltv_c = df.groupby('Customer ID').agg({
    'Invoice': lambda x: x.nunique(),
    'Quantity': lambda x: x.sum(),
    'TotalPrice': lambda x: x.sum()})

cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
cltv_c = cltv_c[(cltv_c["total_price"] > 0)]
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12347.00,2,828,1323.32
12348.00,1,373,222.16
12349.00,4,988,2646.99
12351.00,1,261,300.93
12352.00,2,188,343.80


In [8]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]

cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
Customer ID,,,,,
12347.00,2,828,1323.32,661.66,0.00
12348.00,1,373,222.16,222.16,0.00
12349.00,4,988,2646.99,661.75,0.00
12351.00,1,261,300.93,300.93,0.00
12352.00,2,188,343.80,171.90,0.00


In [9]:
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]

churn_rate = 1 - repeat_rate

cltv_c['profit_margin'] = cltv_c["total_price"] * 0.10

cltv_c['customer_value'] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]

cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102.00,95,122988,341776.73,3597.65,0.02,34177.67,79.82,9718080.96,A
14646.00,87,168501,243853.05,2802.91,0.02,24385.31,56.95,4947113.98,A
14156.00,138,106885,183180.55,1327.40,0.03,18318.06,42.78,2791606.81,A
14911.00,270,66561,137675.91,509.91,0.06,13767.59,32.15,1576926.47,A
13694.00,105,123639,128172.42,1220.69,0.02,12817.24,29.93,1366736.21,A
